In [1]:
%reset -f

In [2]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
print('Google Drive connected.')

Mounted at /content/gdrive
Google Drive connected.


In [3]:
!nvidia-smi

Thu Dec 10 21:05:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import cv2
import numpy as np
import glob

lista_imagenes = glob.glob("/content/gdrive/MyDrive/Remasterizar/output_frames_TecoGAN/*.png")

lista_imagenes.sort()

img = cv2.imread(lista_imagenes[0])
height, width, layers = img.shape
frameSize = (width,height)
print("Resolution: " + str(frameSize))
lista_imagenes[0:10]

Resolution: (1920, 1440)


['/content/gdrive/MyDrive/Remasterizar/output_frames_TecoGAN/output_00001000.png',
 '/content/gdrive/MyDrive/Remasterizar/output_frames_TecoGAN/output_00001001.png',
 '/content/gdrive/MyDrive/Remasterizar/output_frames_TecoGAN/output_00001002.png',
 '/content/gdrive/MyDrive/Remasterizar/output_frames_TecoGAN/output_00001003.png',
 '/content/gdrive/MyDrive/Remasterizar/output_frames_TecoGAN/output_00001004.png',
 '/content/gdrive/MyDrive/Remasterizar/output_frames_TecoGAN/output_00001005.png',
 '/content/gdrive/MyDrive/Remasterizar/output_frames_TecoGAN/output_00001006.png',
 '/content/gdrive/MyDrive/Remasterizar/output_frames_TecoGAN/output_00001007.png',
 '/content/gdrive/MyDrive/Remasterizar/output_frames_TecoGAN/output_00002000.png',
 '/content/gdrive/MyDrive/Remasterizar/output_frames_TecoGAN/output_00002001.png']

In [5]:
out = cv2.VideoWriter('output_video.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 232, frameSize)

for filename in lista_imagenes:
    img = cv2.imread(filename)
    out.write(img)

out.release()

In [6]:
!cp -r /content/output_video.mp4 /content/gdrive/MyDrive/Remasterizar/output_stable_habana_split_dain_tecogan.mp4

In [7]:
# Runtime Type = Python 3
# Hardware Accelerator = GPU 
!git clone https://github.com/jantic/DeOldify.git DeOldify

Cloning into 'DeOldify'...
remote: Enumerating objects: 2187, done.
remote: Total 2187 (delta 0), reused 0 (delta 0), pack-reused 2187
Receiving objects: 100% (2187/2187), 69.39 MiB | 36.36 MiB/s, done.
Resolving deltas: 100% (995/995), done.


In [1]:
cd DeOldify

/content/DeOldify


In [2]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

from os import path

In [3]:
!pip install -r colab_requirements.txt

In [4]:
import fastai
from deoldify.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

In [5]:
!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth # Modelo OK
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth # Modelo Artistico

--2020-12-10 21:53:17--  https://data.deepai.org/deoldify/ColorizeVideo_gen.pth
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874066230 (834M) [application/octet-stream]
Saving to: ‘./models/ColorizeVideo_gen.pth’

./models/ColorizeVi 100%[===================>] 833.57M  19.9MB/s    in 44s     

2020-12-10 21:54:01 (19.0 MB/s) - ‘./models/ColorizeVideo_gen.pth’ saved [874066230/874066230]

--2020-12-10 21:54:01--  https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255144681 (243M) [application/octet-stream]
Saving to: ‘./models/ColorizeArtistic_gen.pth’

./models/ColorizeAr 100%[===================>] 243.32M  20.1MB/s    in 13s     



In [6]:
#!wget https://media.githubusercontent.com/media/jantic/DeOldify/master/resource_images/watermark.png -O ./resource_images/watermark.png

In [7]:
colorizer = get_artistic_video_colorizer()
#colorizer = get_video_colorizer()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [8]:
colorizer.result_folder = Path("/content/gdrive/MyDrive/Remasterizar/DeOldify")

In [10]:
colorizer._colorize_from_path(Path("/content/gdrive/MyDrive/Remasterizar/output_stable_habana_split_dain_tecogan.mp4"),
                              render_factor = 30, #param min: 7, max: 40
                              watermarked=False, post_process = True)

Video created here: /content/gdrive/MyDrive/Remasterizar/DeOldify/output_stable_habana_split_dain_tecogan.mp4


PosixPath('/content/gdrive/MyDrive/Remasterizar/DeOldify/output_stable_habana_split_dain_tecogan.mp4')

In [11]:
colorizer = get_video_colorizer()

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


In [12]:
colorizer.result_folder = Path("/content/gdrive/MyDrive/Remasterizar/DeOldify_video")

In [ ]:
colorizer._colorize_from_path(Path("/content/gdrive/MyDrive/Remasterizar/output_stable_habana_split_dain_tecogan.mp4"),
                              render_factor = 30, #param min: 7, max: 40
                              watermarked=False, post_process = True)